## McKinsey Hackathon

Author: John Lingi <br>
email: jl1809@imperial.ac.uk

##### Problem Description
Your client is an Insurance company and they need your help in building a model to *predict the propensity to pay renewal premium* and *build an incentive plan* for its agents to **maximise the net revenue** (i.e. renewals - incentives given to collect the renewals) collected from the policies post their issuance.

 

You have information about past transactions from the policy holders along with their demographics. The client has provided aggregated historical transactional data like number of premiums delayed by 3/ 6/ 12 months across all the products, number of premiums paid, customer sourcing channel and customer demographics like age, monthly income and area type.

 

In addition to the information above, the client has provided the following relationships:

Expected effort in hours put in by an agent for incentives provided; and
Expected increase in chances of renewal, given the effort from the agent.
 

Given the information, the client wants you to predict the propensity of renewal collection and create an incentive plan for agents (at policy level) to maximise the net revenues from these policies.

### Evaluation Criteria
Your solutions will be evaluated on 2 criteria:

The base probability of receiving a premium on a policy without considering any incentive. The monthly incentives you will provide on each policy to maximize the net revenue. 

Part A:
The probabilities predicted by the participants would be evaluated using AUC ROC score.

Part B:
The monthly incentives you will provide on each policy to maximize the net revenue

### Data Variables

Base Factors
1. percentage premium paid by cash credit: Percentage of premium amount paid by cash or credit card <br> 
2. age in days: policy holder's age <br>
3. income: monthly income of policy holder <br>
4. count 3-6 months late: number of premiums late by 3-6 months <br>
5. count 6-12 months late: number of premiums late by 6-12 months <br>
6. count more than 12 months late: number of premiums late by more than 12 months <br>
7. application underwriting score: applicant score at time of application (if score < 90 then not insured) <br>
8. number of premiums paid: total premiums paid on time until now <br>
9. sourcing channel: where applicants were sourced from <br>
10. residence area type: urban / rural <br>
11. premium amount: monthly premium amount <br>


Added
12. premium to income ratio <br>
13. late payments to on-time payments ratio <br>
14. stickiness: combines age, number of on-time payments and sourcing channel to give an estimate of loyalty of the customer. The sourcing channel is included as I hypothesise customers are more likely to stick around if they are onboarded from non-conventional sources like friend referrals. This has a small weighting though.

Label
15. renewal: policy renewed or not <br>

### Additional Information from web

##### Factors that influence a customer's renewal tendency
source: http://pwc.blogs.com/analytics_means_business/2016/03/price-elasticity-better-ways-of-predicting-insurance-customer-behaviours-.html <br><br>

1. Whether the customer made a claim over the policy year. A positive claims service experience will be in the insurer's favour, while a bad experience is likely to result in the loss of the customer, unless the customer can be swayed - assuming the insurer wants to retain the customer of course. <br><br>

2. If the customer is cash rich, but time poor, the convenience of simply renewing with the insurer for a premium that looks broadly in line with the expiring price (and perhaps only marginally different terms and conditions) may be very attractive. Determining whether a customer fits into this category has historically involved considering the customer’s credit score, and any financial health, socio-economic or demographic information that is available. <br><br>

3. If the customer is motivated by paying the lowest price, and does not care about the quality of service or the value proposition on offer, the renewal premium would need to be pitched aggressively (especially if the customer has already visited price comparison websites to assess the renewal quote) in order to have any chance of retaining the customer. <br><br>

source: EU study on consumers’ decision making in insurance services: a behavioural economics perspective

Behavioural economics: people with a large amount of wealth are more likely to buy insurance than those with less. Not just from a perspective of they can afford it, but also because humans feel "loss" more than gain and those who've gained more have more to lose.

Peace of mind: Consumers may purchase products without giving their decision too much thought for motivations relating to ‘peace of mind’. This is the main driver in decisions leading to overinsurance and to payment of premiums which are too expensive for the cover they offer. This motivation is linked to loss and regret aversion. Consumers may fear a potential loss to an extent that makes them willing to pay a high premium to insure against it. Moreover, they may fear the regret they would feel in case they did not purchase the insurance cover and a claim arose. (Is it possible to get a signal of this from the data?)

Economics:
In insurance purchases, scarcity becomes important when decisions are taken under time pressure, but also in relation to general (money) poverty. Less financially able individuals have been shown to take worse decisions in complex situations compared to the more affluent. For example, it is possible that some individuals choose too high an excess for their financial ability which would put them under pressure if a claim occurs.

Renewals:
Many consumers are laissez-faire about renewals allowing automatic renewals if available. Automatic renewal can benefit consumers as it ensures continuous risk cover. This can be particularly important for mandatory cover such as (motor) third party liability.

The focus groups found that passiveness and inertia, manifested in a tendency for consumers to remain with their current insurer, and in some cases never even consider comparing offers, is a potential source of suboptimal decision making. This is driven by a perception that switching is risky and that it is important to maintain a longstanding relationship with an insurer, and in one country (Romania) even by concerns about the viability of insurers. 

Why people buy insurance:
1. mandatory
2. peace of mind
3. social persuasion - friend / family
4. (renewal specific): they have the best deal

Specific situations that may trigger purchase or renewal of insurance:
1. The focus groups and the literature reveal that life events, such as moving home, or the purchase of a new vehicle, usually trigger insurance purchases. 
2. Occurence of a damage
3. On the advice from a 3rd party

**Sales channels**
A behavioural factor which was identified as being important in influencing consumer behaviour was trust. By nature of the product, a customer will only fully experience the product’s quality in case a claim arises. The customer must therefore trust that the insurer will provide effective cover in case of a claim prior to purchasing the insurance policy.

*Agents*
According to desk research and stakeholders, most consumers seem to conclude their insurance contracts directly with the insurance company, or via intermediaries such as banks, brokers and agents. But this differs across EU member states. Overall, consumers preferred sales channel is via independent intermediaries.

*Online*
The desk research and stakeholder consultations also confirmed that online sales are becoming increasingly popular especially among the younger population. Less than potential in some places.


Source: The rich get richer and the poor get poorer: On risk aversion in behavioral decision-making
Participants who experienced a prior monetary loss displayed more risky choice behavior on the IGT than subjects who experienced a prior gain. These effects were not mediated by a positive or negative affect, although the sample size may have been too small to detect a small effect


Source: Halifax home insurance peace of mind report 2009
There is surely a correlation between the higher Peace of Mind factor in less densely
populated regions

Source: http://www.bain.com/publications/articles/customer-behavior-loyalty-in-insurance-global-2017.aspx
Loyalty

In [31]:
import data_utils

In [32]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import roc_auc_score

from scipy.optimize import minimize
from scipy.stats import boxcox, mode

from xgboost import XGBClassifier

### Prepare data

In [33]:
raw_data = pd.read_csv("train_ZoGVYWq.csv")

In [34]:
processed_data = data_utils.process_data(raw_data=raw_data, upsample={"bool": True, "n":20000})

1    74855
0    20000
Name: renewal, dtype: int64
shape: (94855, 15)


The function 'process data' performs the following:
1. Fills missing data using median values. The columns this is relevant to are the late payment counts and the applicant credit score. We assume no customer has an score less than 90.
2. Upsampling minority class to obtain better signal during training
3. Label encoder sourcing channel and residence area type
4. Create the following new columns: total late payments, premium to income ratio, delayed payment ratio, loyalty
5. Drop id column

## Part A

### Model & Training

In [13]:
x_train, x_test, y_train, y_test = data_utils.split_data(processed_data, test_size=0.2, random_state=1)
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((75884, 14), (75884,), (18971, 14), (18971,))

We tested many models including KNN, Logistic Regression, Feed-forward Neural Networks and Decision Trees. We found the best models to be an ensemble of Gradient Boosted and Random Forest models.

In [7]:
classifier = RandomForestClassifier(n_estimators=100, min_samples_leaf=25)
classifier.fit(x_train, y_train)
probs = classifier.predict_proba(x_test)
roc = roc_auc_score(y_true=y_test, y_score=probs[:, 1])        

In [8]:
"RF roc score:", roc

('roc score:', 0.9201632332310106)

In [12]:
classifier = XGBClassifier(n_estimators=100)
classifier.fit(x_train, y_train)
probs = classifier.predict_proba(x_test)
roc = roc_auc_score(y_true=y_test, y_score=probs[:, 1])

In [13]:
"XGB roc score:", roc

('GBC roc score:', 0.8522587426169247)

#### Create Ensemble

In [37]:
def train_ensemble(x, y, n=7):
    """
    Create ensemble with random parameters
    """
    
    models = []
    for i in range(n):
        print("Creating model:", i + 1)
        
        if i % 2 == 0:
            model = XGBClassifier(max_depth=np.random.randint(3, 10), n_estimators=np.random.randint(100, 150), random_state=1)
        else:
            model = RandomForestClassifier(n_estimators=np.random.randint(100, 150), min_samples_leaf=np.random.randint(30, 50), random_state=1)

        model.fit(x, y)
        models.append(model)
        
    return models

In [36]:
def ensemble_predict(x, models, prediction_shape):
    """
    Uses the mode to predict
    """

    preds = np.zeros(shape=(len(models), prediction_shape))
    for i, model in enumerate(models):
    
        single_model_pred = model.predict_proba(x)
        preds[i, :] = single_model_pred[:, 1] # get the value for class 1
        final_preds = mode(preds)[0]

    return final_preds.squeeze()

In [22]:
models = train_ensemble(x_train, y_train)
final_predictions = ensemble_predict(x_test, models, prediction_shape=y_test.shape[0])
roc_auc_score(y_true=y_test, y_score=final_predictions)

In [23]:
"Ensemble roc score:", roc

('Ensemble roc score:', 0.9293537563560632)

Fit to all data

In [38]:
x, y = data_utils.split_data(processed_data, split_train_test=False)
final_models = train_ensemble(x, y)

Creating model: 1
Creating model: 2
Creating model: 3
Creating model: 4
Creating model: 5
Creating model: 6
Creating model: 7


## Part B

### Calculate incentives for policies

In [39]:
test_data_raw = pd.read_csv("test_66516Ee.csv")

In [40]:
# Location for final table of results
final_results = pd.DataFrame(test_data_raw["id"])

In [41]:
test_data = data_utils.process_data(test_data_raw, upsample={"bool":False})

shape: (34224, 14)


Predict test probabilities

In [42]:
final_results["renewal"] = ensemble_predict(test_data, final_models, prediction_shape=test_data.shape[0])

In [43]:
final_results

,id,renewal
0,649,0.961198
1,81136,0.893452
2,70762,0.434282
3,53935,0.912627
4,15476,0.819777
5,64797,0.947653
6,67412,0.480863
7,44241,0.392992
8,5069,0.961283
9,16615,0.965912


#### Optimiser

In [44]:
def pp(x):
    """
    Calculates percentage increase in renewal probability.
    
    x: incentive
    """
    return 0.2 * (1 - np.exp(-0.2 * (10 * (1 - np.exp(- 0.0025 * x)))))

In [45]:
def revenue(x, premium, renewal_probs):
    """
    Function we are trying to maximise.
    """
    return -1.0 * np.sum(premium * renewal_probs * (1 + pp(x)) - x)

In [46]:
def constraint1(x):
    """ strictly positive incentive"""
    return x

def constraint3(x, renewal_probs):
    """ renewal probabilities cannot be greater than 1"""
    return 1 - renewal_probs * (1 + pp(x))

In [47]:
def vanilla_cb(x):
    print(x)

def maximise_revenue(premium, renewal_probs, x_init, bounds):    
    """
    Optimiser
    """
    result = minimize(fun=revenue, 
                      x0=x_init, 
                      args=(premium, renewal_probs), 
                      method="SLSQP", 
                      bounds=bounds,
                      tol = 1e-4,
                      constraints=({"type":"ineq", "fun": lambda x: constraint1(x)},
                                  {"type":"ineq", "fun": lambda x, renewal_probs=renewal_probs: 
                                   constraint3(x, renewal_probs=renewal_probs)}), 
#                       callback=vanilla_cb,
#                       options={'disp': True, 'eps':1e1, "maxiter":1000}
                     )
    return result

We call the optimiser now. Each policy is independent of the others so we can approach this greedily and try to maximise each one

In [48]:
x_init = 50 * np.random.rand(final_results.shape[0]) # initial values for incentives
bounds = tuple((0, None) for x in x_init) # bounds for incentives

incentives = np.zeros_like(x_init) # array to store final results

premiums = test_data["premium"]
renewals = final_results["renewal"]

# Loop through each policy and maximise
for i in range(len(x_init)):
    premium = premiums[i]
    renewal = renewals[i]
    x0 = np.array([x_init[i]])
    
    # Skip over policies where prob of renewal is maximum
    if renewal != 1:    
        optimisation_results = maximise_revenue(premium, renewal, x0.flatten(), [bounds[i]])
        incentives[i] = optimisation_results.x

print(incentives)

[ 47.84007531 241.69312095 263.83058364 ...   2.37467144 282.63585859
 246.15785069]


Add incentives to final results table and save

In [49]:
final_results["incentives"] = incentives

In [50]:
final_results

,id,renewal,incentives
0,649,0.961198,4.784008e+01
1,81136,0.893452,2.416931e+02
2,70762,0.434282,2.638306e+02
3,53935,0.912627,1.576356e+02
4,15476,0.819777,3.547035e+02
5,64797,0.947653,7.051090e+01
6,67412,0.480863,6.482711e+01
7,44241,0.392992,1.094398e+02
8,5069,0.961283,4.771051e+01
9,16615,0.965912,4.084394e+01


In [51]:
final_results.to_csv("mckinsey_datahack_results_2.csv", index=False)

Thank you!

Note: cross-validation was hyperparameter optimisation were performed in a seperate script